### Notebook which collects code for image manipulations from Xenium data
- **Developed by:** Anna Maguza
- **Place:** Wuerzburg Institute for System Immunology
- **Date:** 26th March 2024

##### Import packages

+ Single cell packages

In [1]:
import anndata as ad
import scanpy as sc
import squidpy as sq
import pandas as pd
import scipy as sci
from scipy.io import mmread
import matplotlib.pyplot as plt
import scipy as sci
from scipy.sparse import coo_matrix

+ Image manipulation packages

In [2]:
import numpy as np
import tifffile as tf
from scipy.ndimage import affine_transform
from PIL import Image, ImageFile
import json

+ Load ome.tiff image

In [4]:
ome_tiff_path = '/../Non-diseased_pre-designed_pallet/Xenium_V1_hColon_Non_diseased_Base_FFPE_he_image.ome.tif'
img = tf.imread(ome_tiff_path)

#### Convert ome.tiff to png

+ Save level 1 (lowres)

In [5]:
png_path = '/../Non-diseased_pre-designed_and_add-on_panel/Xenium_V1_hColon_Non_diseased_Base_FFPE_he_image_level1.png'

In [6]:
img = tf.imread(ome_tiff_path, series=0, level=1)  

if img.dtype != np.uint8:
    img = (255 * (img - img.min()) / (img.ptp())).astype(np.uint8)

Image.fromarray(img).save(png_path)

print(f"Converted level 1 of {ome_tiff_path} to {png_path}")

Converted level 1 of /mnt/LaCIE/annaM/gut_project/raw_data/Xenium_10X_datasets/Gut_samples/Non-diseased_pre-designed_pallet/Xenium_V1_hColon_Non_diseased_Base_FFPE_he_image.ome.tif to /mnt/LaCIE/annaM/gut_project/raw_data/Xenium_10X_datasets/Gut_samples/Non-diseased_pre-designed_and_add-on_panel/Xenium_V1_hColon_Non_diseased_Base_FFPE_he_image_level1.png


+ Save level 0 (hires)

In [7]:
png_path = '/../Non-diseased_pre-designed_and_add-on_panel/Xenium_V1_hColon_Non_diseased_Base_FFPE_he_image_level0.png'
img = tf.imread(ome_tiff_path, series=0, level=0)  

if img.dtype != np.uint8:
    img = (255 * (img - img.min()) / (img.ptp())).astype(np.uint8)

Image.fromarray(img).save(png_path)

print(f"Converted level 1 of {ome_tiff_path} to {png_path}")

Converted level 1 of /mnt/LaCIE/annaM/gut_project/raw_data/Xenium_10X_datasets/Gut_samples/Non-diseased_pre-designed_pallet/Xenium_V1_hColon_Non_diseased_Base_FFPE_he_image.ome.tif to /mnt/LaCIE/annaM/gut_project/raw_data/Xenium_10X_datasets/Gut_samples/Non-diseased_pre-designed_and_add-on_panel/Xenium_V1_hColon_Non_diseased_Base_FFPE_he_image_level0.png


#### Change png 

+ Load png image

In [ ]:
Image.MAX_IMAGE_PIXELS = None
hires = np.asarray(Image.open(png_path))

+ Rotate png image (level - 1)

In [12]:
Image.MAX_IMAGE_PIXELS = None
image_path = '/../Xenium_V1_hColon_Non_diseased_Base_FFPE_he_image_level1.png'
image = Image.open(image_path)

# Rotate the image
rotated_image = image.rotate(-90)  # Rotate 90 degrees

# Save the rotated image
rotated_image_path = '/../Non-diseased_pre-designed_and_add-on_panel/Xenium_V1_hColon_Non_diseased_Base_FFPE_he_image_level1_rotated.png'
rotated_image.save(rotated_image_path)

+ Rotate png image (level - 0)

In [13]:
Image.MAX_IMAGE_PIXELS = None
image_path = '/../Non-diseased_pre-designed_and_add-on_panel/Xenium_V1_hColon_Non_diseased_Base_FFPE_he_image_level0.png'
image = Image.open(image_path)

# Rotate the image
rotated_image = image.rotate(-90)  # Rotate 90 degrees

# Save the rotated image
rotated_image_path = '/../Non-diseased_pre-designed_and_add-on_panel/Xenium_V1_hColon_Non_diseased_Base_FFPE_he_image_level0_rotated.png'
rotated_image.save(rotated_image_path)

In [ ]:
original_image_path = png_path
rotated_image_path = '/path_to_the_image/rotated_image.png'

In [ ]:
!magick convert original_image_path -rotate -90 rotated_image_path

+ Reduce png image size

In [ ]:
import io

hires_image = Image.fromarray(hires)

buf = io.BytesIO()
hires_image.save(buf, format="PNG", quality=40) 
buf.seek(0)

# Load the image back from the buffer
compressed_image = Image.open(buf)


In [ ]:
# Specify the path where you want to save the compressed image
compressed_image_path = '/path_to_the_image/compressed_image.png'  

# Save the compressed image to the specified path
compressed_image.save(compressed_image_path)

+ Create scale_factors.json for Xenium data

In [ ]:
scale_factors_data = {
    'level0': {'pixel_size': 0.2125, 'x_pixels': 35416, 'y_pixels': 25778},
    'level1': {'pixel_size': 0.4250, 'x_pixels': 17708, 'y_pixels': 12889},
    'level2': {'pixel_size': 0.8500, 'x_pixels':  8854, 'y_pixels':  6444},
    'level3': {'pixel_size': 1.7000, 'x_pixels':  4427, 'y_pixels':  3222},
    'level4': {'pixel_size': 3.4000, 'x_pixels':  2213, 'y_pixels':  1611},
    'level5': {'pixel_size': 6.8000, 'x_pixels':  1106, 'y_pixels':   805},
    'level6': {'pixel_size':13.6000, 'x_pixels':   553, 'y_pixels':   402},
    'level7': {'pixel_size':27.2000, 'x_pixels':   276, 'y_pixels':   201}
}

json_str = json.dumps(scale_factors_data, indent=4)

with open('path/scale_factors_data.json', 'w') as f:
    f.write(json_str)

In [ ]:
scale_factors = {"tissue_hires_scalef": 0.000006000112943,
                "tissue_lowres_scalef": 0.000024000451773}

json_str = json.dumps(scale_factors, indent=4)

with open('path/scale_factors.json', 'w') as f:
    f.write(json_str)


#### Manual adding of image to the anndata

+ Load h5ad object

In [5]:
adata_xenium = sc.read_10x_h5('/../Non-diseased_pre-designed_and_add-on_panel/outs/cell_feature_matrix.h5')

+ Load ome.tiff image

In [ ]:
ome_tiff_path = '/path/image.ome.tif'
img = tf.imread(ome_tiff_path)

+ Load coordinates

In [15]:
coords = pd.read_csv("/../cells.csv", header=0)

+ Load nucleus boundaries

In [ ]:
nucleus_boundaries = pd.read_csv("/path/nucleus_boundaries.csv",header=0)

#### Create anndata object

+ Prepare obs

In [8]:
adata_xenium.obs.index.name = 'cell_id'

+ change cell coordinates

In [10]:
coords.head()

,cell_id,x_centroid,y_centroid,transcript_counts,control_probe_counts,control_codeword_counts,unassigned_codeword_counts,deprecated_codeword_counts,total_counts,cell_area,nucleus_area
0,aaaagkoa-1,830.147217,836.563416,284,0,0,0,0,284,653.365805,23.120001
1,aaablbol-1,735.468018,854.389282,116,0,0,0,0,116,651.243461,12.643750
2,aaabogma-1,847.423462,846.633240,170,0,0,0,0,170,674.544087,18.152813
3,aaabpdim-1,1543.696045,4223.798340,61,0,0,0,0,61,172.406569,11.379375
4,aaabpfoj-1,1551.099121,4220.062988,105,0,0,0,0,105,283.490948,56.806565


In [16]:
# rename x_centroid to y_centroid and y_centroid to x_centroid
coords.rename(columns={'x_centroid':'y_centroid', 'y_centroid':'x_centroid'}, inplace=True)
# make x_centroid a first column, y_centroid a second column, leave other columns as they are
coords = coords[['x_centroid', 'y_centroid'] + [col for col in coords.columns if col not in ['x_centroid', 'y_centroid']]]
coords.head()

,x_centroid,y_centroid,cell_id,transcript_counts,control_probe_counts,control_codeword_counts,unassigned_codeword_counts,deprecated_codeword_counts,total_counts,cell_area,nucleus_area
0,836.563416,830.147217,aaaagkoa-1,284,0,0,0,0,284,653.365805,23.120001
1,854.389282,735.468018,aaablbol-1,116,0,0,0,0,116,651.243461,12.643750
2,846.633240,847.423462,aaabogma-1,170,0,0,0,0,170,674.544087,18.152813
3,4223.798340,1543.696045,aaabpdim-1,61,0,0,0,0,61,172.406569,11.379375
4,4220.062988,1551.099121,aaabpfoj-1,105,0,0,0,0,105,283.490948,56.806565


+ Prepare uns

In [12]:
spatial_key = "spatial"
library_id = "tissue42"

In [13]:
adata_xenium.uns["spatial"] = dict()
adata_xenium.uns["spatial"][library_id] = dict()

+ Add coordinates to obsm

In [17]:
adata_xenium.obs = adata_xenium.obs.merge(coords, on='cell_id', how='left')

In [18]:
adata_xenium.obsm["spatial"] = adata_xenium.obs[["x_centroid", "y_centroid"]].copy().to_numpy()

+ Add image to uns

In [19]:
adata_xenium.uns[spatial_key] = {library_id: {}}
adata_xenium.uns[spatial_key][library_id]["images"] = {}
adata_xenium.uns[spatial_key][library_id]["images"] = {"hires": hires}
adata_xenium.uns[spatial_key][library_id]["images"] = {"lowres": lowres}

+ Add scale factors

In [ ]:
adata_xenium.uns['spatial']['spatial']["scalefactors"] = scale_factors

#### Create anndata object with `stlearn` package

In [ ]:
import stlearn as st

In [ ]:
adata = st.ReadXenium(feature_cell_matrix_file="/../Gut_samples/Non-diseased_pre-designed_and_add-on_panel/outs/cell_feature_matrix.h5",
                     cell_summary_file="/.../Gut_samples/Non-diseased_pre-designed_and_add-on_panel/outs/cells.csv",
                     library_id="hColon_Non_diseased_Add_on_FFPE",
                     image_path="/.../Gut_samples/Non-diseased_pre-designed_and_add-on_panel/Xenium_V1_hColon_Non_diseased_Base_FFPE_he_image_level0_rotated.png",
                     scale=3,
                     spot_diameter_fullres=15 # Recommend
                     )